 <div style="text-align:center;">
   <span style="color:green; font-size:2em; font-weight:bold;">Exploratory Data Analysis (EDA)</span><br><br>
</div>


# <span style="color:green; text-align:center;">Introduction</span>

Dans cette partie, nous nous intéressons à la visualisation des données ainsi qu'à l'analyse descriptive du jeu de données **"final_data"** obtenu suite au nettoyage des bases de données initiales du fichier "Préparation_données.ipynb".

Pour les taux de chômage et de croissance, nous allons se concentrer uniquement sur les observations de la période s'étallant de **2019** à **2023**. Nous voulons à partir de cela visualiser les niveaux moyens de chomage et de croissance économique pour les cinq dernières années.
L'objectif ici est de mettre en pratique les méthodes vues en cours.

**Précision :**

Pour la représentation graphique des séries temporelles, la période considérée est celle s'étallant de **1994** à **2023**.

-La visualisation se fera par le biais de deux applications Dash.
-L'analyse descriptive portera sur une série temporelle particulière.

In [ ]:
# Import des packages
from importlib import reload
import declarations as d
reload(d)

In [ ]:
# Importation  de la base de données
df = d.pd.read_csv("../final_data.csv")
df.head()

## <span style="color:green; text-align:center;">I- Présentation de la base</span>

In [ ]:
# Dimensions de la DF
df.shape

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df.info()

In [ ]:
# Générer des statistiques descriptives récapitulatives d'une DataFrame
df.describe()

In [ ]:
# Telécharger le shapefile
world = d.gpd.read_file(d.gpd.datasets.get_path('naturalearth_lowres'))

# Transformer les coordonnées géographiques en coordonnées projetées
world.to_crs('EPSG:4326') 

# Renommer la colonne iso_a3
world=world.rename(columns={'iso_a3': 'COUNTRY'})

In [ ]:
# Visualisation
world.head()

In [ ]:
# Sélectionner certaines colonnes
columns_to_keep = ['geometry', 'name','COUNTRY', 'continent']
world = world[columns_to_keep]

# Fusionner les deux DataFrames
world_df = d.pd.merge(world, df, how='inner', on=['COUNTRY'])

In [ ]:
# Visualisation
world_df.head()

## <span style="color:green; text-align:center;">II- Visualisation des données </span>

In [ ]:
# Plot
world_df.plot(color='lightgrey')

Deux applications seront créées pour la visualisation des données : corrige la phrase.
<div style="margin-left: 20px;">
-La premiere sera instantanné et à l'échelle mondiale. <br>
-La seconde c'est la représentation de la série temporelle par continent.
    </div>

## <span style="color:green; text-align:center;">I-1 APPLICATION 1</span>

In [ ]:
# Créer un Dash app
app1 = d.dash.Dash(__name__)

# Définir le layout 
app1.layout = d.html.Div([
    # Sélecteur d'année
    d.dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in sorted(world_df['YEAR'].unique())],
        value=world_df['YEAR'].min(),  #valeur par defaut de l'année
        multi=False,
    ),
    # Sélecteur de variables
    d.dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'Unemployment Rate', 'value': 'Unemployment_rate'},
            {'label': 'GDP growth Rate', 'value': 'GDP_rate'},
            {'label': 'Life expentancy', 'value': 'life_expentancy'},
            {'label': 'Population growth rate', 'value': 'pop_growth_rate'}
          
        ],
        value='Unemployment_rate',  # valeur par defaut de la variable representée
        multi=False,
    ),
    d.dcc.Graph(id='choropleth-map'),
])

# Définir un callback  pour mettre à jour la carte sur l'année selectionnée 
@app1.callback(
    d.Output('choropleth-map', 'figure'),
    [d.Input('year-dropdown', 'value'),
     d.Input('variable-dropdown', 'value')]
)
def update_map(selected_year, selected_variable):
    subset_gdf = world_df[world_df['YEAR'] == selected_year]
    subset_gdf.index = subset_gdf['name']
    fig = d.px.choropleth(
        subset_gdf,
        geojson=subset_gdf.geometry,
        locations=subset_gdf.index,
        color=selected_variable,
        projection="natural earth",
        title=f'{selected_variable} in {selected_year}',
    )
    fig.update_geos(fitbounds="locations", visible=True)
  
    fig.update_layout ( margin = { "r" : 0 , "t" : 0 , "l" : 0 , "b" : 0 }) 
    # Afficher le graphique interactif
    #fig.show()
    return fig

# Executer l'application
if __name__ == '__main__':
    app1.run_server(debug=True, port=8050)


## <span style="color:green; text-align:center;">I-2 APPLICATION 2: visualisations des series temporelles par contient</span>

In [ ]:
app2 = d.dash.Dash(__name__)

# Créer votre graphique initial
fig = d.px.line(world_df, x='YEAR', y=["Unemployment_rate"], color='name')

# Créer la mise en page de votre application Dash
app2.layout = d.html.Div([
    # Sélecteur de continents
    d.dcc.Dropdown(
        id='continent-selector',
        options=[
            {'label': continent, 'value': continent}
            for continent in world_df['continent'].unique()
        ],
        multi=True,
        value=world_df['continent'].unique()  # Sélectionner tous les continents par défaut
    ),
    # Sélecteur de variables
    d.dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'Unemployment Rate', 'value': 'Unemployment_rate'},
            {'label': 'GDP growth Rate', 'value': 'GDP_rate'},
            {'label': 'Life expentancy', 'value': 'life_expentancy'},
            {'label': 'Population growth rate', 'value': 'pop_growth_rate'}
          
        ],
        value='Unemployment_rate',  # Valeur par defaut de la variable representée
        multi=False,
    ),
    
    # Graphique
    d.dcc.Graph(
        id='line-chart',
        figure=fig
    )
])

# Définisser la logique de la mise à jour du graphique en fonction de la sélection du sélecteur
@app2.callback(
    d.Output('line-chart', 'figure'),
    [d.Input('variable-dropdown', 'value'),
     d.Input('continent-selector', 'value')]
)
def update_graph2(selected_variable, selected_continents):
    filtered_df = world_df[world_df['continent'].isin(selected_continents)]
    fig = d.px.line(filtered_df, x='YEAR', y=[selected_variable], color='name',
                   labels={'YEAR': 'Année', selected_variable : f"{selected_variable}"})
    
    
    return fig


# Exécuter l'application Dash
if __name__ == '__main__':
    app2.run_server(debug=True, port=8051)


## <span style="color:green; text-align:center;">I-3 APPLICATION 3: visualisations de la tendance moyenne des </span>

In [ ]:
# sélectionner uniquement les 4 colonnes et faire la moyenne dans chaque groupe
df_group = world_df.groupby('name')[
    ['Unemployment_rate', 'GDP_rate', 'life_expentancy', 'pop_growth_rate']
    ].mean().reset_index()
df_group.head()

In [ ]:
app3 = d.dash.Dash(__name__)

# Créer votre graphique initial
fig = d.px.bar(df_group, x='name', y='Unemployment_rate', color='name',
             labels={'name': 'Pays', 'Unemployment_rate': 'Taux de chômage moyen'},
             template='plotly_dark')

# Créer la mise en page de votre application Dash
app3.layout = d.html.Div([
    
    # Sélecteur de variables
    d.dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'Unemployment Rate', 'value': 'Unemployment_rate'},
            {'label': 'GDP growth Rate', 'value': 'GDP_rate'},
            {'label': 'Life expentancy', 'value': 'life_expentancy'},
            {'label': 'Population growth rate', 'value': 'pop_growth_rate'}
          
        ],
        value='Unemployment_rate',  # valeur par defaut de la variable representée
        multi=False,
    ),
    
    # Graphique
    d.dcc.Graph(
        id='bar',
        figure=fig
    )
])

# Définisser la logique de la mise à jour du graphique en fonction de la sélection du sélecteur
@app3.callback(
    d.Output('bar', 'figure'),
    [d.Input('variable-dropdown', 'value')]
)
def update_graph3(selected_variable):
    #fig = px.line(filtered_df, x='YEAR', y=["Unemployment_rate"], color='COUNTRY')
    fig = d.px.bar(df_group, x='name', y=[selected_variable], color='name',
                   labels={'name': 'Pays', selected_variable: selected_variable},
                   template='plotly_dark')
    # Afficher le graphique interactif
    
    return fig


# Exécuter l'application Dash
if __name__ == '__main__':
    app3.run_server(debug=True, port=8052)


# <span style="color:green; text-align:center;">III- Analyse descriptive </span>

In [ ]:
final_data= d.pd.read_csv('C:/Projet-Python/Projet-Python/final_data.csv')

In [ ]:
final_data.head()

## <span style="color:green; text-align:center;">II-1.Série temporelle taux de chômage </span>

In [ ]:
# un exemple

# Utilisation de la fonction avec le DataFrame 'final_data', le pays 'AUS', et la variable 'Unemployment_rate'
analyse_serie_temporelle_pays(world_df,'GDP_rate', 'AUS')


In [ ]:
# exemple 2
analyse_serie_temporelle_pays(world_df,'Unemployment_rate', 'PHL')



par suite , nous allons implementer la fonction 'analyse_serie_temporelle_pays' dans une application dash

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

df=world_df.copy()


app = dash.Dash(__name__)



# Liste des pays disponibles dans le dataframe
pays_disponibles = df['name'].unique()

app.layout = html.Div([
    html.H1("Analyse de Série Temporelle"),
    
    
   # Sélecteur de variables
    d.dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'Unemployment Rate', 'value': 'Unemployment_rate'},
            {'label': 'GDP growth Rate', 'value': 'GDP_rate'},
            {'label': 'Life expentancy', 'value': 'life_expentancy'},
            {'label': 'Population growth rate', 'value': 'pop_growth_rate'}
          
        ],
        value='Unemployment_rate',  # valeur par defaut de la variable representée
        multi=False,
    ),
    
    
    # Sélection du pays
    html.Label("Choisissez le pays :"),
    dcc.Dropdown(
        id='dropdown-pays',
        options=[
            {'label': pays, 'value': pays} for pays in pays_disponibles
        ],
        value=pays_disponibles[0]
    ),
    
    # Graphiques
    dcc.Graph(id='time-series-plot'),
    dcc.Graph(id='decomposition-plot'),
])

@app.callback(
    [Output('time-series-plot', 'figure'),
     Output('decomposition-plot', 'figure')],
    [Input('variable-dropdown', 'value'),
     Input('dropdown-pays', 'value')]
)
def update_graph(indicateur, pays):
    # Sélectionner la série temporelle du pays spécifique
    serie_temporelle = df[indicateur].dropna()
    
    # Moyenne mobile d'ordre 4
    rolling_mean = serie_temporelle.rolling(window=4).mean()

    # Créer le premier graphique (série temporelle et moyenne mobile)
    time_series_fig = px.line(x=serie_temporelle.index, y=[serie_temporelle, rolling_mean],
                              labels={'variable': 'Type', 'value': 'Valeur'},
                              title=f'Série Temporelle et Moyenne Mobile - {indicateur} en {pays}',
                              color_discrete_map={'0': 'blue', '1': 'red'})
    
    # Décomposition saisonnière
    decomposition = seasonal_decompose(serie_temporelle, model='multiplicative', period=4)

    # Créer le deuxième graphique (décomposition)
    decomposition_fig = px.line(x=serie_temporelle.index, y=[decomposition.trend, decomposition.seasonal, decomposition.resid],
                                labels={'variable': 'Composant', 'value': 'Valeur'},
                                title=f'Décomposition - {indicateur} en {pays}',
                                color_discrete_map={'0': 'green', '1': 'orange', '2': 'gray'})

    return time_series_fig, decomposition_fig

if __name__ == '__main__':
    app.run_server(debug=True)